No entanto, o algoritmo que é frequentemente usado em otimização de portfólios é o Algoritmo Genético (GA).
O GA é amplamente adotado na área de finanças quantitativas devido à sua capacidade de lidar com problemas complexos e não lineares,
como a seleção de ativos em um portfólio. Ele tem a flexibilidade de ser ajustado para atender às necessidades específicas de um investidor 
em termos de restrições, preferências de risco e retorno.

- Tive problemas com o Acesso à API, porque eu tava pegando uma API Premium ('TIME_SERIES_DAILY_ADJUSTED')
- Usei Thundercliente para testar API

In [2]:
import numpy as np
from deap import base, creator, tools, algorithms
import pandas as pd
import requests

# Função para calcular o retorno e o risco de um portfólio
def evaluate_portfolio(individual, data):
    weights = np.array(individual)
    returns = np.sum(data.pct_change().mean() * weights) * 252
    volatility = np.sqrt(np.dot(weights.T, np.dot(data.pct_change().cov() * 252, weights)))
    return returns, volatility

# Função para obter os dados de preços de fechamento de ativos da Alpha Vantage
def get_alpha_vantage(symbols, api_key):
    data = {}

    for symbol in symbols:
        base_url = 'https://www.alphavantage.co/query'
        params = {
            'function': 'TIME_SERIES_DAILY',
            'symbol': symbol,
            'apikey': api_key
        }

        try:
            response = requests.get(base_url, params=params)
            raw_data = response.json()
            time_series = raw_data.get('Time Series (Daily)', {})

            # Armazena os dados em um dicionário
            data[symbol] = {date: float(info['4. close']) for date, info in time_series.items()}
        except Exception as e:
            print(f"Erro ao obter dados para {symbol}: {e}")

    return data

def main():
    # Símbolos dos ativos que você deseja incluir no portfólio
    assets = ["AAPL", "GOOGL", "MSFT", "AMZN", "TSLA"]
    n_assets = len(assets)

    # Substitua 'YOUR_API_KEY' pela sua chave de API Alpha Vantage
    api_key = '4KR9RHTR37YQ1XY5'

    # Obtém os dados de preços de fechamento dos ativos
    asset_data = get_alpha_vantage(assets, api_key)

    # Cria um DataFrame a partir dos dados obtidos
    data = pd.DataFrame(asset_data)

    # Defina os parâmetros do algoritmo genético
    creator.create("FitnessMulti", base.Fitness, weights=(1.0, -1.0))  # Maximizar retorno e minimizar risco
    creator.create("Individual", list, fitness=creator.FitnessMulti)

    toolbox = base.Toolbox()
    toolbox.register("attr_float", np.random.uniform, 0, 1)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=n_assets)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", evaluate_portfolio, data=data)
    toolbox.register("mate", tools.cxBlend, alpha=0.5)
    toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.2, indpb=0.2)
    toolbox.register("select", tools.selNSGA2)

    # Crie a população inicial
    population = toolbox.population(n=100)

    # Execute o algoritmo genético
    n_generations = 50

    for gen in range(n_generations):
        offspring = algorithms.varAnd(population, toolbox, cxpb=0.7, mutpb=0.2)
        fits = toolbox.map(toolbox.evaluate, offspring)

        for fit, ind in zip(fits, offspring):
            ind.fitness.values = fit

        population = toolbox.select(offspring, len(population))

    # Recupere a solução ótima (portfólio)
    best_portfolio = tools.selBest(population, 1)[0]

    print("Melhor alocação de ativos:")
    print(assets)
    print(best_portfolio)
    best_return, best_risk = evaluate_portfolio(best_portfolio, data)
    print(f"Retorno anual: {best_return * 100:.2f}%")
    print(f"Risco anual: {best_risk * 100:.2f}%")

if __name__ == "__main__":
    main()


Melhor alocação de ativos:
['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'TSLA']
[5.592947866141986, 1.2527674066366972, -1.345318515758397, -2.1775764024889788, 1.5083585938875794]
Retorno anual: 167.14%
Risco anual: 148.78%
